# Cleaning the Mental Health Dataset
### Importing the Dataset:

In [1]:
library(tidyverse)
library(readxl)
library(dplyr)
library(glue)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘glue’


The following object is masked from ‘package:dplyr’:

    collapse




## Step1 :Creating functions to read 3 different typeset Excel files

> ##### Since the collected dataset has 3 different collation patterns, I 
created three different functions to read them, all of them basically do the same job, they read different page numbers, rename DHB names to match other files, rename columns, etc.

In [3]:
# Function to read excel sheets and extract columns & rows of Canterbury DHB for the years of 2004 - 2008
read_excel_mh <- function(file){
    path <- glue('Mental health data/{file}')
    colheads <- read_excel(path, sheet = 4, skip = 4) %>% names()
    colheads[2] <- 'Sex'
    colheads[1] <- 'DHB'
    
    year <- file %>%substr(6, 9) # Extract year from filename
    
    excel <- read_excel(path, sheet=4, col_names = colheads)
    
    northland <- excel %>% slice(7:9) %>% mutate(DHB = 'Northland District Health Board')
    waitemata <- excel %>% slice(11:13) %>% mutate(DHB = 'Waitemata District Health Board')
    auckland <- excel %>% slice(15:17) %>% mutate(DHB = 'Auckland District Health Board')
    manukau <- excel %>% slice(19:21) %>% mutate(DHB = 'Counties Manukau District Health Board')
    ccdhb <- excel %>% slice(55:57) %>% mutate(DHB = 'Capital and Coast District Health Board')
    huttvalley <- excel %>% slice(59:61) %>% mutate(DHB = 'Hutt Valley District Health Board')
    wairarapa <- excel %>% slice(63:65) %>% mutate(DHB = 'Wairarapa District Health Board')
    canterbury <- excel %>% slice(75:77) %>% mutate(DHB = 'Canterbury District Health Board')
    s.canterbury <- excel %>% slice(79:81) %>% mutate(DHB = 'South Canterbury District Health Board')
    
    df <- northland %>%
        rbind(waitemata, auckland, manukau, ccdhb, huttvalley, wairarapa, canterbury, s.canterbury) %>%
        mutate(Year = year) %>%
        select(DHB, Sex, Total, Year)
    names(df) <- c('DHB', 'sex', 'total', 'year')
    
    return(df)
}

# Function to read excel sheets and extract columns & rows of Canterbury DHB for the year 2009-2010
read_excel_mh2 <- function(file){
    path <- glue('Mental health data/{file}')
    df <- read_excel(path, sheet = 12, col_names =c('DHB', 'Total', 'Rates')) %>%
    select(1:2) %>%
    slice(3:25) %>%
    filter(grepl('Northland|Waitemata|Auckland|Manukau|Capital|Hutt|Wairarapa|Canterbury', DHB)) %>%
    mutate(Year = '2010', DHB = paste0(DHB, ' District Health Board'), Sex = 'Total')
    names(df) <- c('DHB', 'total', 'year', 'sex')
        
    return(df)
}

# Function to read excel sheets and extract columns & rows of Canterbury DHB for the year 2010-2020
read_excel_mh3 <- function(file, sheetnum){

    path <- glue('Mental health data/{file}')
    df <- read_excel(path, sheet = sheetnum, skip = 3) %>%
    select(1:3)
    
    names(df) <- c('DHB', 'sex', 'total') # Rename columns
    yr <- file %>%substr(6, 9) # Extract year from filename
    
    df <- df %>% mutate(DHB = if_else(is.na(DHB), lag(DHB), DHB), year = yr) %>%
            fill(DHB) %>%
            filter(grepl('Northland|Waitemata District|Waitematā|Auckland District|Manukau District|Capital|Hutt|Wairarapa District|Canterbury District', DHB)) %>%
            mutate(DHB = if_else(grepl("Waitematā",DHB), 'Waitemata District Health Board', DHB))
    
    return(df)
}

#### read all the files

In [5]:
# Read the columns of Canterbury DHB
mh_df_0405 <- read_excel_mh("01 mh2005.xls")
mh_df_0506 <- read_excel_mh("02 mh2006.xls")
mh_df_0607 <- read_excel_mh("03 mh2007.xls")
mh_df_0708 <- read_excel_mh("04 mh2008.xls")
# Year 0809 is missing
mh_df_0910 <- read_excel_mh2("06 mh2010.xls")
mh_df_1011 <- read_excel_mh3("07 mh2011.xlsx", 14)
mh_df_1112 <- read_excel_mh3("08 mh2012.xlsx", 15)
mh_df_1213 <- read_excel_mh3("09 mh2013.xlsx", 11)
mh_df_1314 <- read_excel_mh3("10 mh2014.xlsx", 11)
mh_df_1415 <- read_excel_mh3("11 mh2015.xlsx", 11)
mh_df_1516 <- read_excel_mh3("12 mh2016.xlsx", 11)
mh_df_1617 <- read_excel_mh3("13 mh2017.xlsx", 11) %>% slice(-c(28, 29))
mh_df_1718 <- read_excel_mh3("14 mh2018.xlsx", 11) %>% slice(-c(28, 29))
mh_df_1819 <- read_excel_mh3("15 mh2019.xlsx", 11)
mh_df_1920 <- read_excel_mh3("16 mh2020.xlsx", 11)


New names:
* `` -> ...1
* `` -> ...2

New names:
* `` -> ...1
* `` -> ...2

New names:
* `` -> ...1
* `` -> ...2

New names:
* `` -> ...1
* `` -> ...2

New names:
* `` -> ...1
* `` -> ...2
* `` -> ...3

New names:
* `` -> ...1
* `` -> ...2
* `` -> ...3

New names:
* `` -> ...1

New names:
* `` -> ...1

New names:
* `` -> ...1

New names:
* `` -> ...1

New names:
* `` -> ...1
* `` -> ...2
* `` -> ...3

New names:
* `` -> ...1
* `` -> ...2
* `` -> ...3

New names:
* `` -> ...3

New names:
* `` -> ...1
* `` -> ...2
* `` -> ...3



##### An example showing the properties it contains

In [7]:
mh_df_0405 %>% head()

DHB,sex,total,year
<chr>,<chr>,<chr>,<chr>
Northland District Health Board,Total,3306,2005
Northland District Health Board,Male,1827,2005
Northland District Health Board,Female,1479,2005
Waitemata District Health Board,Total,15028,2005
Waitemata District Health Board,Male,8437,2005
Waitemata District Health Board,Female,6591,2005


## Step2 :Merge all data sets

In [9]:
# Bind the tables into one dataframe (note: 2010 only has total number of patients)
mh_df <- rbind(mh_df_0405,
              mh_df_0506,
              mh_df_0607,
              mh_df_0708,
              mh_df_0910,
              mh_df_1011,
              mh_df_1112,
              mh_df_1213,
              mh_df_1314,
              mh_df_1415,
              mh_df_1516,
              mh_df_1617,
              mh_df_1718,
              mh_df_1819,
              mh_df_1920)

In [10]:
# save as csv
write.csv(mh_df, "mental_health_by_dhb_RAW.csv", row.names=TRUE) # Save raw csv file for DHB's data

## Step3: Summarize the data set
   > ##### Group DHBs into a region they belong to, or close to. 
   
   > ##### Auckland & Northland : Northland DHB, Waitemata DHB, Auckland DHB, Manukau DHB
   
   > ##### Wellington : Capital and Coast DHB, Hutt Valley DHB, Wairarapa DHB
   
   > ###### Canterbury : Canterbury DHB, South Canterbury DHB

In [11]:
city <- function(df){
    df<- df %>% mutate(DHB = if_else(grepl("Northland|Waitemata|Auckland|Manukau",DHB), 'Auckland', DHB)) %>%
          mutate(DHB = if_else(grepl("Capital|Wairarapa|Hutt",DHB), 'Wellington', DHB)) %>%
          mutate(DHB = if_else(grepl("Canterbury",DHB), 'Canterbury', DHB)) %>%
          mutate(total = total %>% as.integer) %>%
          group_by(DHB, year, sex) %>% summarise(total = sum(total))
    
    colnames(df)[1] = 'city'
    
    return(df)
}

mh_df2 <- city(mh_df)
mh_df2

`summarise()` has grouped output by 'DHB', 'year'. You can override using the `.groups` argument.



city,year,sex,total
<chr>,<chr>,<chr>,<int>
Auckland,2005,Female,15635
Auckland,2005,Male,17251
Auckland,2005,Total,32886
Auckland,2006,Female,16427
Auckland,2006,Male,18497
Auckland,2006,Total,34925
Auckland,2007,Female,17918
Auckland,2007,Male,20522
Auckland,2007,Total,38441


In [53]:
# save as csv
write.csv(mh_df2, "FINAL_mental_health_by_city.csv", row.names=TRUE) # Save csv file for each city by year